In [7]:
#https://gist.github.com/nicolewhite/167828e51d8f2b6fad75
!pip install tweepy 
!pip install unidecode
!pip install unicodecsv


import tweepy
from tweepy import Cursor
import unicodecsv
from unidecode import unidecode


from credentials import *

# Authentication and connection to Twitter API.

auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Usernames whose tweets we want to gather.
users = ["realDonaldTrump"]

with open('trump_replies_4.csv', 'wb') as file:
    writer = unicodecsv.writer(file, delimiter = ',', quotechar = '"')
    # Write header row.
    writer.writerow(["name",
                    "username",
                    "followers_count",
                    "listed_count",
                    "following",
                    "favorites",
                    "verified",
                    "default_profile",
                    "location",
                    "time_zone",
                    "statuses_count",
                    "description",
                    "geo_enabled",
                    "contributors_enabled",
                    "tweet_id",
                    "tweet_time",
                    "tweet_text",
                    "tweet_lat",
                    "tweet_long",
                    "tweet_source",
                    "tweet_in_reply_to_screen_name",
                    "tweet_direct_reply",
                    "tweet_retweet_status",
                    "tweet_retweet_count",
                    "tweet_favorite_count",
                    "tweet_hashtags",
                    "tweet_hashtags_count",
                    "tweet_urls",
                    "tweet_urls_count",
                    "tweet_user_mentions",
                    "tweet_user_mentions_count",
                    "tweet_media_type",
                    "tweet_contributors"])

    for user in users:
        user_obj = api.get_user(user)

        # Gather info specific to the current user.
#         user_info = [user_obj.name,
#                      user_obj.screen_name,
#                      user_obj.followers_count,
#                      user_obj.listed_count,
#                      user_obj.friends_count,
#                      user_obj.favourites_count,
#                      user_obj.verified,
#                      user_obj.default_profile,
#                      user_obj.location,
#                      user_obj.time_zone,
#                      user_obj.statuses_count,
#                      user_obj.description,
#                      user_obj.geo_enabled,
#                      user_obj.contributors_enabled]

        # Get 1000 most recent tweets for the current user.
#         api.search, q="to:"+user  api.user_timeline, screen_name = user

        for otweet in Cursor(api.user_timeline, screen_name = user).items(15):
           
        
            for tweet in Cursor(api.search, q="to:"+user, max_id = otweet.id).items(100):
                # Retweets start with "RT ..."
                retweet_status = True if tweet.text[0:3] == "RT " else False
                if (not retweet_status):

                    user_info = [tweet.user.name,
                             tweet.user.screen_name,
                             tweet.user.followers_count,
                             tweet.user.listed_count,
                             tweet.user.friends_count,
                             tweet.user.favourites_count,
                             tweet.user.verified,
                             tweet.user.default_profile,
                             tweet.user.location,
                             tweet.user.time_zone,
                             tweet.user.statuses_count,
                             tweet.user.description,
                             tweet.user.geo_enabled,
                             tweet.user.contributors_enabled]





                    # Latitude and longitude stored as array of floats within a dictionary.
                    lat = tweet.coordinates['coordinates'][1] if tweet.coordinates != None else None
                    long = tweet.coordinates['coordinates'][0] if tweet.coordinates != None else None
                    # If tweet is not in reply to a screen name, it is not a direct reply.
                    direct_reply = True if tweet.in_reply_to_screen_name != "" else False
        #             # Retweets start with "RT ..."
        #             retweet_status = True if tweet.text[0:3] == "RT " else False

                    # Get info specific to the current tweet of the current user.
                    tweet_info = [tweet.in_reply_to_status_id_str,
                                  tweet.created_at,
                                  unidecode(tweet.text),
                                  lat,
                                  long,
                                  tweet.source,
                                  tweet.in_reply_to_screen_name,
                                  direct_reply,
                                  retweet_status,
                                  tweet.retweet_count,
                                  tweet.favorite_count]

                    # Below entities are stored as variable-length dictionaries, if present.
                    hashtags = []
                    hashtags_data = tweet.entities.get('hashtags', None)
                    if(hashtags_data != None):
                        for i in range(len(hashtags_data)):
                            hashtags.append(unidecode(hashtags_data[i]['text']))

                    urls = []
                    urls_data = tweet.entities.get('urls', None)
                    if(urls_data != None):
                        for i in range(len(urls_data)):
                            urls.append(unidecode(urls_data[i]['url']))

                    user_mentions = []
                    user_mentions_data = tweet.entities.get('user_mentions', None)
                    if(user_mentions_data != None):
                        for i in range(len(user_mentions_data)):
                            user_mentions.append(unidecode(user_mentions_data[i]['screen_name']))

                    media = []
                    media_data = tweet.entities.get('media', None)
                    if(media_data != None):
                        for i in range(len(media_data)):
                            media.append(unidecode(media_data[i]['type']))

                    contributors = []
                    if(tweet.contributors != None):
                        for contributor in tweet.contributors:
                            contributors.append(unidecode(contributor['screen_name']))

                    more_tweet_info = [', '.join(hashtags),
                                       len(hashtags),
                                       ', '.join(urls),
                                       len(urls),
                                       ', '.join(user_mentions),
                                       len(user_mentions),
                                       ', '.join(media),
                                       ', '.join(contributors)]

                    # Write data to CSV.
                    writer.writerow(user_info + tweet_info + more_tweet_info)

        # Show progress.
        print("Wrote tweets by %s to CSV." % user)

Wrote tweets by realDonaldTrump to CSV.
